In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
# from mock.tasks import adiciona_carro
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
session.execute("USE simulacao")

In [3]:
session.execute("select count(*) from simulacao")[0]

/tmp/ipykernel_2851/795349805.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  session.execute("select count(*) from simulacao")[0]


Row(count=3071)

In [4]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 142 ALLOW FILTERING;")

In [5]:
result[0]

/tmp/ipykernel_2851/223460175.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  result[0]


Row(placa='COL1R23', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=655.0, pos_y=868.0)

In [6]:
r = list(result)

In [7]:
r

[Row(placa='COL1R23', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=655.0, pos_y=868.0),
 Row(placa='CHI0G12', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=385.0, pos_y=198.05),
 Row(placa='CHI2E34', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=745.0, pos_y=774.15),
 Row(placa='URU2Y34', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=655.0, pos_y=258.4999999999992),
 Row(placa='BRA0K12', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=475.0, pos_y=218.45000000000005),
 Row(placa='ECU8G90', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=475.0, pos_y=146.25),
 Row(placa='URU6M78', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=475.0, pos_y=100.30000000000001),
 Row(placa='BOL3R45', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=745.0, pos_y=780.0),
 Row(placa='GUY7J89', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=655.0, pos_y=720.15),
 Row(placa='ECU2U34', rodovia='BR-116', tempo_da_simulacao=142.0, pos_x=475.0, pos_y=96.85),
 Row(placa='URU8C90', rodovi

In [8]:
df = ss.createDataFrame(r)

df.show()

+-------+-------+------------------+-----+------------------+
|  placa|rodovia|tempo_da_simulacao|pos_x|             pos_y|
+-------+-------+------------------+-----+------------------+
|COL1R23| BR-116|             142.0|655.0|             868.0|
|CHI0G12| BR-116|             142.0|385.0|            198.05|
|CHI2E34| BR-116|             142.0|745.0|            774.15|
|URU2Y34| BR-116|             142.0|655.0| 258.4999999999992|
|BRA0K12| BR-116|             142.0|475.0|218.45000000000005|
|ECU8G90| BR-116|             142.0|475.0|            146.25|
|URU6M78| BR-116|             142.0|475.0|100.30000000000001|
|BOL3R45| BR-116|             142.0|745.0|             780.0|
|GUY7J89| BR-116|             142.0|655.0|            720.15|
|ECU2U34| BR-116|             142.0|475.0|             96.85|
|URU8C90| BR-116|             142.0|475.0|170.60000000000002|
|BOL9D01| BR-116|             142.0|835.0|             711.0|
|PER5V67| BR-116|             142.0|745.0|             784.0|
|CHI6S78

In [9]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 143 ALLOW FILTERING;")
r = list(result)
df2 = ss.createDataFrame(r)
df2.show()

ValueError: can not infer schema from empty dataset

In [ ]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 144 ALLOW FILTERING;")
r = list(result)
df25 = ss.createDataFrame(r)
df25.show()

In [ ]:
df_empty = ss.createDataFrame([], "placa: string, posicao: int, linha: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean, tempo_em_curso: int")

In [ ]:
import json
params = json.load(open('./mock/parametros.json'))

In [ ]:
p = [[key]+list(params[key].values()) for key in params.keys()]

In [ ]:
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))

p.toPandas()

In [ ]:
import queue


In [ ]:
fps = 30
vel_media = 0
n_vel_media = 0

tempo_medio = 0
n_tempo_medio = 0

def atualiza_media(media_atual, tamanho_atual, media_add, tamanho_add):
    if media_add == None:
        return media_atual
    if tamanho_atual == 0:
        return media_add
    tamanho_total = tamanho_atual + tamanho_add
    return (media_atual/tamanho_total)*tamanho_atual + (media_add/tamanho_total)*tamanho_add

def processa_velocidade_media(batch):
    global vel_media, n_vel_media
    batch = batch.na.fill(0, subset=['velocidade'])
    vel_media_batch = batch.agg(F.mean('velocidade')).collect()[0][0]
    length_batch = batch.select(F.count('velocidade')).collect()[0][0]
    n_vel_media += length_batch
    vel_media = atualiza_media(vel_media, n_vel_media, vel_media_batch, length_batch)
    pass

def processa_tempo_cruzamento(batch):
    global n_tempo_medio, tempo_medio
    batch = batch.filter(col("tempo_em_curso") != 0)
    tempo_medio_batch = batch.agg(F.mean('tempo_em_curso')).collect()[0][0]
    length_batch = batch.select(F.count('tempo_em_curso')).collect()[0][0]
    n_tempo_medio += length_batch
    tempo_medio = atualiza_media(tempo_medio, n_tempo_medio, tempo_medio_batch, length_batch)
    pass

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    DadosCarros = DadosCarros.drop('aplicaMulta')

    # Renomeia coluna do dado novo
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia').alias('rodovia_nova'),
                                   F.col('pos_x').alias('linha_nova'),
                                   F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')
    
    CarrosSumidos = DadosCarros.join(DadosNovos, on="placa", how='left_anti')
    processa_tempo_cruzamento(CarrosSumidos)
    data_joined = data_joined.withColumn("tempo_inicio", when(col("tempo_da_simulacao").isNull(), col("tempo_da_simulacao_novo")).otherwise(col("tempo_da_simulacao")))
    data_joined = data_joined.withColumn("tempo_em_curso", col("tempo_da_simulacao_novo") - col("tempo_inicio"))
    data_joined = data_joined.withColumn("rodovia", coalesce(col("rodovia_nova"), col("rodovia")))
    data_joined = data_joined.withColumn("linha", coalesce(col("linha_nova"), col("linha")))
    data_joined = data_joined.drop("rodovia_nova", "linha_nova")
    
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)

    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")

    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))
    processa_velocidade_media(data_joined)
    
    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))

    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))

    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")

    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")

    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")

    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))

    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )

    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')

    data_joined = data_joined.drop("VelocidadeMaxima")
    return data_joined
    

In [ ]:
df_empty.show()

In [ ]:
df3 = processa_carro(df, df_empty, 1, 0.5, p)

In [ ]:
df3 = processa_carro(df2, df3, 1, 0.5, p)

In [ ]:
df4 = processa_carro(df25, df3, 1, 0.5, p)

In [ ]:
df4.show()

In [ ]:
vel_media

In [ ]:
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window

window_spec = Window.partitionBy("rodovia", "linha").orderBy('posicao')

# Use lag function with the window specification
lag_column = col("velocidade")*(col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec))

# Add the lag column to the DataFrame
result = df4.withColumn("lag_value", when(lag_column < 0, 1).otherwise(0))

# Show the result
result.toPandas()

In [ ]:
from pyspark.sql.window import Window

# Sample data
data = [("Alice", 25),
        ("Bob", 30),
        ("Charlie", 27),
        ("Diana", 35),
        ("Eva", 32)]

df10.withColumn("RowNumber", F.row_number().over(windowSpec)).show()